In [0]:
df = spark.sql('select * from retail_cat.silver_sch.orders_silver')
df.display()

In [0]:
df_dimcust = spark.sql('select DimCustomerkey,customer_id as dim_customer_id from retail_cat.gold_Sch.dimcustomers')
df_dimprod = spark.sql('select product_id as DimProductkey,product_id as dim_product_id from retail_cat.gold_Sch.dimproducts')

In [0]:
df_fact =  df.join("df_dimcust",df.customer_id == df_dimcust.dim_customer_id,'left').join("df_dimprod",df.product_id == df_dimprod.dim_product_id,how='left')
df_fact_new = df_fact.drop('dim_customer_id','dim_product_id','customer_id','product_id')
df_fact_new.display()

## UPSERT ON FACT

In [0]:
from delta.tables import DeltaTable

In [0]:
if spark.catalog.tableExists("retail_cat.gold_sch.factorders"):

    dlt_obj = DeltaTable.forName(spark,"retail_cat.gold_sch.factorders")
    dlt_obj.alias("target").merge(df_fact_new.alias("source"),"target.order_id = source.order_id").whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()

else:
    
    df_fact_new.write.format("delta")\
        .option("path","abfss://gold@adlscardatalake.dfs.core.windows.net/factorders")\
        .saveAsTable("retail_cat.gold_sch.factorders")